# Introduction to portfolio optimization
## Example: momentum strategy

---
 
 
- Copyright (c) Lukas Gonon, 2024. All rights reserved

- Author: Lukas Gonon <l.gonon@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.9

Disclaimer: this notebook does not consitute any kind of investment advice. It merely serves as an illustrative example.

We start by importing relevant packages.

In [ ]:
import yahoo_fin.stock_info as yf  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import random
import sklearn
from sklearn.linear_model import LinearRegression

#### Construct list of tickers and download data

In [ ]:
### Here we download the data from yahoo finance. Alternatively, see below, we can load it directly from a csv file. 

# Download the data 
# Specify the data range in American format

start_date = "01/01/2011"
end_date = "12/31/2023"
tickers1 = ["AMD","AMZN","AXP","AZO","CLX","DLTR", "DVN", "FCX", "GOOG", "JPM", "KO", "MMM","MSFT","NFLX","MRO"]

tickers2 = ['MMM', 'ABT', 'ABBV', 'ACN', 'AYI', 'ADBE', 'AMD', 'AAP', 'AES', 'AET', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANDV', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'AJG', 'AIZ', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BWA', 'BXP', 'BSX', 'BHF', 'BMY', 'AVGO', 'BF.B', 'CHRW', 'CA', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'CBOE', 'KMX', 'CCL', 'CAT', 'CBG', 'CBS', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CHK', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'GLW', 'COST', 'COTY', 'CCI', 'CSRA', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DOV', 'DWDP', 'DPS', 'DTE', 'DRE', 'DUK', 'DXC', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ETR', 'EVHC', 'EOG', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ES', 'RE', 'EXC', 'EXPE', 'EXPD', 'ESRX', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FISV', 'FLIR', 'FLS', 'FLR', 'FMC', 'FL', 'F', 'FTV', 'FBHS', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GD', 'GE', 'GGP', 'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GS', 'GT', 'GWW', 'HAL', 'HBI', 'HOG', 'HRS', 'HIG', 'HAS', 'HCA', 'HCP', 'HP', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HPQ', 'HUM', 'HBAN', 'HII', 'IDXX', 'INFO', 'ITW', 'ILMN', 'IR', 'INTC', 'ICE', 'IBM', 'INCY', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IQV', 'IRM', 'JEC', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM', 'JNPR', 'KSU', 'K', 'KEY', 'KMB', 'KIM', 'KMI', 'KLAC', 'KSS', 'KHC', 'KR', 'LB', 'LLL', 'LH', 'LRCX', 'LEG', 'LEN', 'LUK', 'LLY', 'LNC', 'LKQ', 'LMT', 'L', 'LOW', 'LYB', 'MTB', 'MAC', 'M', 'MRO', 'MPC', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MAT', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'KORS', 'MCHP', 'MU', 'MSFT', 'MAA', 'MHK', 'TAP', 'MDLZ', 'MON', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MYL', 'NDAQ', 'NOV', 'NAVI', 'NTAP', 'NFLX', 'NWL', 'NFX', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NBL', 'JWN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'ORLY', 'OXY', 'OMC', 'OKE', 'ORCL', 'PCAR', 'PKG', 'PH', 'PDCO', 'PAYX', 'PYPL', 'PNR', 'PBCT', 'PEP', 'PKI', 'PRGO', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'RL', 'PPG', 'PPL', 'PX', 'PCLN', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RRC', 'RJF', 'RTN', 'O', 'RHT', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'COL', 'ROP', 'ROST', 'RCL', 'CRM', 'SBAC', 'SCG', 'SLB', 'SNI', 'STX', 'SEE', 'SRE', 'SHW', 'SIG', 'SPG', 'SWKS', 'SLG', 'SNA', 'SO', 'LUV', 'SPGI', 'SWK', 'SBUX', 'STT', 'SRCL', 'SYK', 'STI', 'SYMC', 'SYF', 'SNPS', 'SYY', 'TROW', 'TPR', 'TGT', 'TEL', 'FTI', 'TXN', 'TXT', 'TMO', 'TIF', 'TWX', 'TJX', 'TMK', 'TSS', 'TSCO', 'TDG', 'TRV', 'TRIP', 'FOXA', 'FOX', 'TSN', 'UDR', 'ULTA', 'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UTX', 'UHS', 'UNM', 'VFC', 'VLO', 'VAR', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VIAB', 'V', 'VNO', 'VMC', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'HCN', 'WDC', 'WU', 'WRK', 'WY', 'WHR', 'WMB', 'WLTW', 'WYN', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XL', 'XYL', 'YUM', 'ZBH', 'ZION', 'ZTS']


nbTickers = len(tickers2)
nbExtract = 100 ## Number of tickers to consider
listExtractTickers = tickers1
listTemp = random.sample(tickers2, nbExtract)
for i in range(nbExtract):
    listExtractTickers.append(listTemp[i]) ## select nbExtract out of the whole list
print("List of extracted tickers: ", listExtractTickers)

raw_tickers = listExtractTickers

## Use the above code to for a random subset of tickers. To use all of them: use the below one.

raw_tickers = tickers2

# Remove duplicates

raw_tickers = list(dict.fromkeys(raw_tickers))

In [ ]:
## Get all tickers from S&P 500.

raw_tickers2 = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
raw_tickers2 = list(raw_tickers2['Symbol'])

## Complement the two lists.

in_first = set(raw_tickers)
in_second = set(raw_tickers2)

in_second_but_not_in_first = in_second - in_first

raw_tickers = raw_tickers + list(in_second_but_not_in_first)

len(raw_tickers)

In [ ]:
in_second_but_not_in_first

In [ ]:
## Check if the data is available for the given ticker for the entire time horizon. Otherwise remove ticker from the list.

df = pd.DataFrame()#yf.get_data(tickers[0], start_date=start_date, end_date=end_date)['adjclose'])
tickers = []
for ticker in raw_tickers:
    print(ticker)
    try: 
        df0 = yf.get_data(ticker, start_date=start_date, end_date=end_date)['adjclose']
        #df0.rename(ticker)
        #print(df0.columns)
        df = pd.concat([df, df0], axis=1)
        tickers.append(ticker)
    except:
        raw_tickers.remove(ticker)
df.columns = tickers
print(len(tickers))

In [ ]:
df.head()

In [ ]:
data = df.dropna('columns')

# Print the first few rows of the data as well as information 
print(data.head())
print(data.info())

In [ ]:
missingtickers=list(df.columns[df.isna().any()])
tickers = [ticker for ticker in tickers if ticker not in missingtickers]
print(missingtickers)
len(tickers)

In [ ]:
# Adjusted closing price for a given ticker and the chosen dates
get_px = lambda x: yf.get_data(x, start_date=start_date, end_date=end_date)['close']#['adjclose']

# Download the data for the tickers in the list
data = pd.DataFrame({sym:get_px(sym) for sym in tickers})
data = data.dropna('columns')

# Print the first few rows of the data as well as information 
print(data.head())
print(data.info())

In [ ]:
## Get closing prices of downloaded data in array
prices = data[tickers]
plot_ix=15
prices.info()
prices.head()

In [ ]:
## Alternatively, we can directly load the data from a csv file using this command:

#prices = pd.read_csv('sp500stocks.csv',index_col=0)
#prices.info()
#prices.head()

# This command could be used to save the data as a csv. 
#prices.to_csv('sp500stocks.csv')

In [ ]:
## Plot prices for the first few tickers
plot_prices = (prices/np.array(prices)[0,:])[tickers[:plot_ix]]
plot_prices.plot(figsize = (15, 6),title='Prices of' + str(plot_ix) + 'S&P500 stocks',ylabel='Adjusted closing price');
plt.show()


In [ ]:
log_ret = np.log(prices/prices.shift(1)).dropna()*252
log_ret

## Cross-sectional momentum strategy

Consider a cross-sectional momentum strategy:

- Sort stocks by their performance in lookback window.
- “Buy winner and sell losers”.
- Simplest implementation: equal weighted portfolio of 5% of most successful stocks, financed by shorting equal weighted portfolio of worst performers.
- Long short portoflio hedges out market risk.
- Averaging over several stocks leads to more stable performance.
- See Jegadeesh and Titman (1993) for more details and a comprehensive study.

In [ ]:
## Choose how many stocks to short and long at each rebalancing step
factors = log_ret
n_titles = int(0.05*len(tickers))
print(n_titles)

The following code implements the cross-sectional momentum strategy. It works as follows: we start with fixed initial capital. Then we compute the moving averages of all factors (in this case log-returns) over a pre-specified horizon. We rebalance at a given frequency (say every 20 trading dates). At each of the rebalancing dates, first we clear our position (sell all the stocks we hold, buy back those we borrowed). This leaves us with a certain amount of money (total_cap).  Then we look at the moving averages of past factors for each stock. Then we choose those stocks that performed best on average (i.e. with the highest moving average) and we buy these n_titles stocks. Similarly we short the n_titles stocks with the worst performance in the recent past (lowest moving average).  The positions we take in these stocks are equally weighted.

In [ ]:
def value_cross_sec_momentum(horizon,frequency,shift=0):
    total_cap = 1000000.        # Initial capital
    value = [total_cap]     # Keep track of the value of your portfolio
    moving_averages = factors.rolling(horizon).mean().dropna()   # Construct moving averages with window size = horizon
    n_dates = int(factors.shape[0]/frequency)  # Compute number of dates for rebalancing
    dates = factors.index.values      # Get dates from dataframe
    rebalancing_dates = [dates[shift+(i+1)*frequency] for i in range(n_dates)]  # Compute dates at which portfolio is rebalanced
    units_old = np.zeros([len(tickers)])   # Initialize "old units" to 0
    set_cap = False   # Set to False at the beginning
    for i in range(len(rebalancing_dates)):
        ## We don't trade, if we don't have enough data about past performance available yet
        if (i+1)*frequency<horizon:
            continue
        date = rebalancing_dates[i] # Current date
        rel_ma = moving_averages.loc[date,tickers]  #Access current moving averages
        prices_date = prices.loc[date] #Access current prices
        sort_ind = np.argsort(rel_ma) # Sort stocks according to past performance
        long_ind = sort_ind[len(tickers)-n_titles:] # Indices of those stocks that performed best
        short_ind = sort_ind[:n_titles] # Indices of those stocks that performed worst
        units = np.zeros_like(rel_ma) # Initialize units to 0
        ## In the first iteration this will be false; we start with initial capital. Afterwards this is how much our portfolio is still worth
        if set_cap is True:
            total_cap = value[-1]+np.sum(units_old*prices_date) # Previous value + gains you make from selling stocks (or buying back) you bought (or sold) in previous period at current price
        units[long_ind] = total_cap/(2*n_titles) # Set equal weights for stocks that you buy
        units[short_ind] = -total_cap/(2*n_titles) # Set equal weights for stocks that you shortsell
        units = units/prices_date # Convert from proportion of wealth to actual units
        ## Update value: liquidate previous position, build current one. 
        value.append(value[-1]+(np.sum(units_old*prices_date)-np.sum(units*prices_date)))
        ## Set variables for next iteration
        units_old = units
        set_cap = True
    ## At terminal time we liquidate the full position:
    value.append(value[-1]+(np.sum(units*prices.loc[dates[-1]])))
    
    ## For benchmarking we may want to create some plots / get some output
    #plt.plot(value,label=str(horizon))
    #print(rebalancing_dates[0])
    #print(rebalancing_dates[-3:])
    return value


How do we measure the performance? A commonly used measure is the Sharpe ratio. This ratio compares the expected return from a portfolio to its risk. The Sharpe ratio is defined as 
$$ \frac{E[R]-R_f}{Std(R)} $$
where $E[R]$ is the expected return of the portfolio, $R_f$ is the risk-free rate, and $Std(R)$ is the standard deviation of returns.
Let us compute the Sharpe ratio of our strategy for a choice of rebalancing frequency and time-window of moving averages.

In [ ]:
frequency = 50
horizon = 100
values = np.array(value_cross_sec_momentum(horizon,frequency))
returns_strat = (values[1:] - values[:-1])/values[:-1]
mu_hat = np.mean(returns_strat)
sigma_hat = np.std(returns_strat)
## returns and std are estimated based on a frequency 20/252 -> annualized sharpe ratio has factor np.sqrt(252/20)
print(mu_hat/sigma_hat*np.sqrt(252/frequency))

Let's compare the performance for varying lookback periods.

In [ ]:
frequency = 20 ## rebalance every 50 trading days
for horizon in [100+i*10 for i in range(8)]:
    print(horizon)
    values = np.array(value_cross_sec_momentum(horizon,frequency))
    returns_strat = (values[1:] - values[:-1])/values[:-1]
    mu_hat = np.mean(returns_strat)
    sigma_hat = np.std(returns_strat)
    ##  annualized sharpe ratio has factor np.sqrt(252(frequency))
    print(mu_hat/sigma_hat*np.sqrt(252/frequency))

Let's now look at a fixed lookback period, but shift the day on which we start the strategy. 

In [ ]:
horizon=130
sharpe=[]
for shift in range(15):
    #total_cap = 1000000.
    values = np.array(value_cross_sec_momentum(horizon,50,shift))
    returns_strat = (values[1:] - values[:-1])/values[:-1]
    mu_hat = np.mean(returns_strat)
    sigma_hat = np.std(returns_strat)
    ## returns and std are estimated based on a frequency 20/252 -> annualized sharpe ratio has factor np.sqrt(252/20)
    sharpe.append(mu_hat/sigma_hat*np.sqrt(252/frequency))
    #print(mu_hat/sigma_hat)
    #print(values[-1])
    #plt.legend()
print(np.mean(sharpe))
print(sharpe)

We see that the performance of this strategy seems to heavily depend on the day we started! Starting one day later affects also all subsequent rebalancing dates. Thus, the different shifts lead to quite a big variation of the final portfolio performance.

Further questions: 
-  How sensitive is this to the choice of the 5% cutoff and the
lookback window?
- How much are the returns reduced due to transaction costs of
1%, 0.5%, or 0.1%?
- Can the strategy be improved by considering combinations of
multiple lookback horizons?
- Or does this just lead to datamining and bad out of sample
performance?

# Time-series momentum strategy based on predicted returns

Next, we aim to examine a second type of strategies based on predicted returns. We follow the same approach as above. But instead of moving averages we now use a prediction model that, based on past and current data, predicts the average returns over the next 15 days. Then again we rank the stocks according to this indicator. We buy those that our model predicts to perform best and sell those stocks that the model predicts to perform worst.

We start by a function that creates lagged time series from our data.

In [ ]:
def create_lagged_data_averages(data, window):
    x, y = [], []
    for i in range(len(data)-windowsize):
        feature = data[i:i+windowsize,:]
        target = np.mean(data[i+windowsize:min(i+windowsize+15,len(data)),:len(tickers)],axis=0)
        x.append(feature)
        y.append(target)
    return np.array(x), np.array(y)

In [ ]:
windowsize = 50
num_train_samples = int(log_ret.shape[0]*0.65)
x_train, y_train = create_lagged_data_averages(np.array(log_ret)[:num_train_samples,:], windowsize)
x_test, y_test = create_lagged_data_averages(np.array(log_ret)[num_train_samples:,:], windowsize)
print('Shapes of training features and targets:', x_train.shape, y_train.shape)
print('Shapes of test features and targets:', x_test.shape, y_test.shape)

First we adapt the strategy above to this more general "predictor". 

In [ ]:
n_dates = len(factors.index.values)

def value_strategy_predictor(horizon,frequency,predictor,shift=0,end_ind=n_dates-1):
    total_cap = 1000000.        # Initial capital
    value = [total_cap]     # Keep track of the value of your portfolio
    n_dates = int(factors.shape[0]/frequency) # compute number of dates for rebalancing
    dates = factors.index.values # Get dates from dataframe
    ## Compute dates at which portfolio is rebalanced. We now also allow for an earlier end.
    rebalancing_dates = [dates[min(shift+(i+1)*frequency,end_ind)] for i in range(n_dates)]
    rebalancing_dates = list(dict.fromkeys(rebalancing_dates)) # Remove any duplicates from the list of rebalancing dates
    units_old = np.zeros([len(tickers)])   # Initialize "old units" to 0
    set_cap = False   # Set to False at the beginning
    for i in range(len(rebalancing_dates)):
      ## We don't trade, if we don't have enough data about past performance available yet
        if (i+1)*frequency<horizon:
            continue
        date = rebalancing_dates[i] # Current date
        ind = int(np.where(dates==date)[0]) # Access index of current date
        inputs = np.array(factors.loc[dates[ind-horizon+1:ind+1]]) # Access past factors in a window from "horizon" up to today
        pred_date = predictor(inputs) # Make predictions about future performance based on todays input
        prices_date = prices.loc[date] #Access current prices
        sort_ind = np.argsort(pred_date) # Sort stocks according to predicted performance
        long_ind = sort_ind[len(tickers)-n_titles:] # Indices of those stocks that performed best
        short_ind = sort_ind[:n_titles] # Indices of those stocks that performed worst
        units = np.zeros([len(tickers)]) # Initialize units to 0
        ## In the first iteration this will be false; we start with initial capital. Afterwards this is how much our portfolio is still worth
        if set_cap is True:
            total_cap = value[-1]+np.sum(units_old*prices_date) # Previous value + gains you make from selling stocks (or buying back) you bought (or sold) in previous period at current price
        units[long_ind] = total_cap/(2*n_titles) # Set equal weights for stocks that you buy
        units[short_ind] = -total_cap/(2*n_titles) # Set equal weights for stocks that you shortsell
        units = units/prices_date # Convert from proportion of wealth to actual units
        ## Update value: liquidate previous position, build current one. 
        value.append(value[-1]+(np.sum(units_old*prices_date)-np.sum(units*prices_date)))
        ## Set variables for next iteration
        units_old = units
        set_cap = True
    ## At terminal time we liquidate the full position:      
    value.append(value[-1]+(np.sum(units*prices.loc[dates[end_ind]])))
    #plt.plot(value,label=str(horizon))
    return value



Now let us build the predictor. The features now consist of past average returns for each of the stocks. 

In [ ]:
x_train_mean = np.mean(np.array(x_train),axis=1)
reg_mean = LinearRegression(fit_intercept=True).fit(x_train_mean, y_train)

print(reg_mean.score(x_train_mean, y_train))
print(sklearn.metrics.mean_squared_error(y_train,reg_mean.predict(x_train_mean)))

x_test_mean = np.mean(np.array(x_test),axis=1)
print(reg_mean.score(x_test_mean, y_test))
print(sklearn.metrics.mean_squared_error(y_test,reg_mean.predict(x_test_mean)))


In [ ]:
def linear_predictor_mean(inputs):
    return reg_mean.predict(np.mean(inputs,axis=0,keepdims=True))[0,:]

In [ ]:
sharpe=[]
out_of_sample_ind = y_train.shape[0]
for shift in range(9):
    values = np.array(value_strategy_predictor(windowsize,frequency,linear_predictor_mean,shift+out_of_sample_ind))
    returns_strat = (values[1:] - values[:-1])/values[:-1]
    mu_hat = np.mean(returns_strat)
    sigma_hat = np.std(returns_strat)
    ## returns and std are estimated based on a frequency 20/252 -> annualized sharpe ratio has factor np.sqrt(252/20)
    sharpe.append(mu_hat/sigma_hat*np.sqrt(252/frequency))
print(np.mean(sharpe))
print(sharpe)

These seem to work well, but again depends on the day on which we start... And again similar questions arise:

-  How sensitive is this to the choice of the 5% cutoff, the
lookback window and the rebalancing frequency?
- How much are the returns reduced due to transaction costs of
1%, 0.5%, or 0.1%?
- Can the strategy be improved by considering combinations of
multiple lookback horizons?
- Or does this just lead to datamining and bad out of sample
performance?

For example: let us look at daily rebalancing instead:

In [ ]:
sharpe=[]
out_of_sample_ind = y_train.shape[0]
for shift in range(9):
    values = np.array(value_strategy_predictor(windowsize,1,linear_predictor_mean,shift+out_of_sample_ind))
    returns_strat = (values[1:] - values[:-1])/values[:-1]
    mu_hat = np.mean(returns_strat)
    sigma_hat = np.std(returns_strat)
    ## returns and std are estimated based on a frequency 20/252 -> annualized sharpe ratio has factor np.sqrt(252/20)
    sharpe.append(mu_hat/sigma_hat*np.sqrt(252/frequency))
print(np.mean(sharpe))
print(sharpe)

Rebalancing every second day:

In [ ]:
sharpe=[]
out_of_sample_ind = y_train.shape[0]
for shift in range(9):
    values = np.array(value_strategy_predictor(windowsize,2,linear_predictor_mean,shift+out_of_sample_ind))
    returns_strat = (values[1:] - values[:-1])/values[:-1]
    mu_hat = np.mean(returns_strat)
    sigma_hat = np.std(returns_strat)
    ## returns and std are estimated based on a frequency 20/252 -> annualized sharpe ratio has factor np.sqrt(252/20)
    sharpe.append(mu_hat/sigma_hat*np.sqrt(252/frequency))
print(np.mean(sharpe))
print(sharpe)

For further reading and more details: see, for example, the textbook "Efficiently Inefficient"
    https://www.jstor.org/stable/j.ctt1287knh

Fintech attempts:

-  Machine learning for finding better "predictors"
- Using machine learning to efficiently select portfolios in presence of high transaction costs
- ...